In [272]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1530680802615_0013,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1530680802615_0013,spark,idle,Link,Link,✔


# Set variables

In [273]:
// The Kafka broker hosts and topic used to write to Kafka
val kafkaBrokers="wn0-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn2-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092"
val kafkaTopic="rawevents"
val zookeepers = "zk1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181,zk3-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181"

zookeepers: String = zk1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181,zk3-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181

In [52]:
val df1 = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("startingOffsets", "earliest").
load()

val query = df1.select(from_json(col("key").cast("Int")), from_json(col("value").cast("string")) )
.map{ case (_, value) => value.split(',') }
.map(s => (s(0).toInt, s(1).toInt,s(2).toDouble)) 



<console>:30: error: overloaded method value from_json with alternatives:
  (e: org.apache.spark.sql.Column,schema: String,options: java.util.Map[String,String])org.apache.spark.sql.Column <and>
  (e: org.apache.spark.sql.Column,schema: org.apache.spark.sql.types.DataType)org.apache.spark.sql.Column <and>
  (e: org.apache.spark.sql.Column,schema: org.apache.spark.sql.types.StructType)org.apache.spark.sql.Column <and>
  (e: org.apache.spark.sql.Column,schema: org.apache.spark.sql.types.DataType,options: java.util.Map[String,String])org.apache.spark.sql.Column <and>
  (e: org.apache.spark.sql.Column,schema: org.apache.spark.sql.types.StructType,options: java.util.Map[String,String])org.apache.spark.sql.Column <and>
  (e: org.apache.spark.sql.Column,schema: org.apache.spark.sql.types.DataType,options: scala.collection.immutable.Map[String,String])org.apache.spark.sql.Column <and>
  (e: org.apache.spark.sql.Column,schema: org.apache.spark.sql.types.StructType,options: scala.collection.immu

In [98]:
%%bash
hdfs dfs -ls /path/checkpointloc

Found 4 items
drwxr-xr-x   - livy supergroup          0 2018-07-04 11:32 /path/checkpointloc/commits
-rw-r--r--   1 livy supergroup         45 2018-07-04 11:32 /path/checkpointloc/metadata
drwxr-xr-x   - livy supergroup          0 2018-07-04 11:33 /path/checkpointloc/offsets
drwxr-xr-x   - livy supergroup          0 2018-07-04 11:32 /path/checkpointloc/sources


In [ ]:
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType};

val df1 = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("group.id", "my-group-5").
option("startingOffsets", "earliest").
load()

                                 
var query = df1.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)").as[(String,String)].
select(col("key"),col("value")).
writeStream.
format("console").
option("failOnDataLoss","false").
outputMode("append").
start()


In [255]:
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType};

val df1 = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("group.id", "my-group-2").
option("startingOffsets", "earliest").
load()

                                 
val query = df1.selectExpr("CAST(key AS STRING)", "CAST(value AS string)").as[(String, String)].
select(col("key"),col("value")).
withColumn("cols",split(col("value"),",")).
select(
  $"cols".getItem(0).as("HouseHoldId"),
  $"cols".getItem(1).as("Timestamp").cast("timestamp"),
  $"cols".getItem(2).as("Value")
).
withWatermark("Timestamp","1 second").
groupBy("Timestamp").
count().
alias("count").
select("Timestamp","count").
writeStream.
format("console").
option("failOnDataLoss","false").
outputMode("append").
start()


query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2dba0855

In [271]:
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType};

val df1 = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("enable.auto.commit","false")
option("startingOffsets", "earliest").

load()

var query = df1.selectExpr("CAST(value AS STRING)").as[(String)].
select(col("value").cast("string")).
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointloc").
option("topic", "cleanseevents").
option("failOnDataLoss","false").
outputMode("append").
start()


An error was encountered:
Invalid status code '404' from http://hn0-sidspa.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:8998/sessions/5 with error payload: "Session '5' not found."


In [127]:
df1.printSchema()


# List Topics

In [260]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --list --zookeeper "zk1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181,zk3-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181"


__consumer_offsets
cleanseevents
rawevents


# Create topic cleansevents

In [46]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic cleanseevents --zookeeper "zk1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181,zk3-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181"

Created topic "cleanseevents".


In [34]:
var ds = df1.select("CAST(key AS Int)",from_json(col("value").cast("string").split(",")).
                    

org.apache.spark.sql.AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
kafka
  at org.apache.spark.sql.catalyst.analysis.UnsupportedOperationChecker$.org$apache$spark$sql$catalyst$analysis$UnsupportedOperationChecker$$throwError(UnsupportedOperationChecker.scala:297)
  at org.apache.spark.sql.catalyst.analysis.UnsupportedOperationChecker$$anonfun$checkForBatch$1.apply(UnsupportedOperationChecker.scala:36)
  at org.apache.spark.sql.catalyst.analysis.UnsupportedOperationChecker$$anonfun$checkForBatch$1.apply(UnsupportedOperationChecker.scala:34)
  at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:127)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$foreachUp$1.apply(TreeNode.scala:126)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$foreachUp$1.apply(TreeNode.scala:126)
  at scala.collection.immutable.List.foreach(List.scala:381)
  at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.sc

In [279]:
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType};

val df1 = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "__consumer_offsets").
option("startingOffsets", "earliest").
load()

var query = df1.selectExpr("CAST(value AS STRING)").as[(String)].
select(col("value").cast("string")).
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointloc").
option("topic", "cleanseevents").
option("failOnDataLoss","false").
outputMode("append").
start()


query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@65f56b13

## List of consumer groups

In [270]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-consumer-groups.sh  --bootstrap-server "wn0-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn2-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092" --list 

spark-kafka-source-4fabfbd7-07bc-4741-aef4-81a798e6e871-1330853161-driver-0
spark-kafka-source-da2f0179-081d-4999-86c3-c7b72d15a978--1471152131-driver-0
spark-kafka-source-781f8643-e840-4ec6-8604-f9743cbad184--1229818353-driver-0


## Delete consumer group

In [142]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-consumer-groups.sh --zookeeper "zk1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181,zk3-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:2181" --delete --group console-consumer-95359

Delete for group console-consumer-95359 failed because group does not exist.


## describe group

In [268]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-consumer-groups.sh  --bootstrap-server "wn0-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn2-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092" --describe  --group  "spark-kafka-source-4fabfbd7-07bc-4741-aef4-81a798e6e871-1330853161-driver-0"

GROUP                          TOPIC                          PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             OWNER
spark-kafka-source-4fabfbd7-07bc-4741-aef4-81a798e6e871-1330853161-driver-0 rawevents                      0          unknown         24812354        unknown         consumer-71_/10.0.0.22


In [156]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-consumer-groups.sh  --bootstrap-server "wn0-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn1-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092,wn2-kafka.vnl0vzclrykufj5kxd5qbv4pse.cx.internal.cloudapp.net:9092" --create --group "raweventreader"

Exception in thread "main" joptsimple.UnrecognizedOptionException: create is not a recognized option
	at joptsimple.OptionException.unrecognizedOption(OptionException.java:108)
	at joptsimple.OptionParser.handleLongOptionToken(OptionParser.java:449)
	at joptsimple.OptionParserState$2.handleArgument(OptionParserState.java:56)
	at joptsimple.OptionParser.parse(OptionParser.java:381)
	at kafka.admin.ConsumerGroupCommand$ConsumerGroupCommandOptions.<init>(ConsumerGroupCommand.scala:446)
	at kafka.admin.ConsumerGroupCommand$.main(ConsumerGroupCommand.scala:46)
	at kafka.admin.ConsumerGroupCommand.main(ConsumerGroupCommand.scala)
